In [51]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [52]:
def send_email(mes):
    
    fromaddr = "hanhanyang313@gmail.com"
    toaddr = "hanhanyang313@gmail.com"

    msg = MIMEMultipart()
    msg["From"] = fromaddr
    msg["To"] = toaddr
    msg["Subject"] = "US Exchange Rate"

    body = mes.to_html(index=False)
    msg.attach(MIMEText(body, 'html'))

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()

    server.login(fromaddr, "juuqmyvlylueykcu")
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    server.quit()

In [55]:
driver = webdriver.Chrome()


while True:
    
    info_data=[]
    
    driver.get("https://www.skbank.com.tw/skbank_web/data?action=rate&rateType=ExchangeRate&v=1621087553160")
    soup = BeautifulSoup(driver.page_source)
    info = soup.select('pre')

    for i in info:
        data = json.loads(i.text)
        USD_datas = data["tableJSON"]["table"]["row"]["data"][0]
    #     print(len(USD_datas))

        for index in range(0,len(USD_datas)):
    #         print(USD_datas[index]["itemContent"])
            if index != 0 and index != 4 and index != 5:
                info_data.append(USD_datas[index]["itemContent"].strip())

    content= pd.DataFrame(np.array(info_data).reshape(1,4),columns=["幣別", "銀行收", "銀行賣","時間"])
    
    
    if(float(content["銀行收"])<28.5 or float(content["銀行賣"])<27.9):
        send_email(content)
    
    sleep(60)
    driver.refresh()
    
driver.close()

KeyboardInterrupt: 